In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import sys
import h5py
import pandas as pd

import strawb
import strawb.sensors.camera as camera
import os
import matplotlib.pyplot as plt

# Load files from the ONC server
Be careful, depending on the amount of data this can take a while!!

## Version 1: Manual version
If file name is known. If not use the database DB

In [ ]:
# onc_downloader = strawb.ONCDownloader(showInfo=False)

# # download files selected by `date_from`, `date_to`,...
# onc_downloader.download_structured(extensions='hdf5',
#                                    date_from = '2021-07-01T23:00:00.000Z',
#                                    date_to = '2021-07-01T23:59:59.000Z',
#                                    dev_codes ='TUMPMTSPECTROMETER002',
#                                    dataProductCode = 'MSSCD',  # short for 'dataProductName': 'Mini Spectrometer Sensor Camera Data',
#                                   )

In [ ]:
# # where data is located on the LRZ VM
# basepath = join(strawb.Config.raw_data_dir, 'tumpmtspectrometer001/2021_09/')
# file_name = os.path.join(basepath, 'TUMPMTSPECTROMETER001_20210904T230000.000Z-SDAQ-CAMERA.hdf5')


# cam_run = camera.FileHandler(file_name)
# picture_handler = camera.Images(cam_run)

## Version 2 (recommended): use the database (DB) to find a valid file

In [ ]:
# load DB
db = strawb.SyncDBHandler(file_name='Default')  # loads the db
db.load_onc_db_update(save_db=True)  # update the DB, could take some time if it has to load info. from ONC

Print some info from the DB

In [ ]:
print(db.dataframe.columns)

### these are the available device codes
print(db.dataframe.deviceCode.unique())

### different measurement types for PMTSPEC and LIDAR
#print(db.dataframe.measurement_type.unique())

### these are the parts of each module that produce data
print(db.dataframe.dataProductCode.unique())

### Select (a) file(s) of interest

In [ ]:
mask = (db.dataframe.deviceCode == 'TUMPMTSPECTROMETER001') # that's the pmtspec module
mask &= (db.dataframe.dataProductCode =='MSSCD') # that's the camera data of the pmtspec module

## select 1h frame around the biolumi event
timestamp = np.datetime64('2021-09-04T23:44:09')
mask &= db.dataframe.dateFrom >= pd.Timestamp(np.datetime64(timestamp, "h"), tz="UTC")
mask &= db.dataframe.dateFrom < pd.Timestamp(np.datetime64(timestamp, "h"), tz="UTC") + np.timedelta64(1, "h")

### selected one file from the DB (it's the same as the file we selected above by hand)
db.dataframe[mask]

### Download the missing files which aren't synced so far from `db.dataframe[mask]`

In [ ]:
if not db.dataframe.synced[mask].all():
    db.update_db_and_load_files(
        db.dataframe[mask],
        output=True,  # print output to console
        download=True,  # download the files
        #save_db=True
    )  # update the DB

In [ ]:
### select only the filename
file_name = db.dataframe.fullPath[mask].iloc[-1]
print(file_name)
camera = strawb.sensors.camera.Camera(file_name)
cam_run = strawb.sensors.camera.FileHandler(file_name)
picture_handler = strawb.sensors.camera.Images(cam_run)

In [ ]:
# print some parameters
print(f'Module: {camera.file_handler.module}')
print(f'Number of Frames: {camera.file_handler.exposure_time.shape[0]}')
print(f'Date: {np.min(camera.file_handler.time.asdatetime()[:])} - {np.max(camera.file_handler.time.asdatetime()[:])}')
print(f'Exposure Times [s]: {np.unique(camera.file_handler.exposure_time)}')

In [ ]:
# mask over a threshold + mask invalid frames + mask no lucifer enabled
mask = (camera.images.integrated_minus_dark > 1e6) & camera.images.invalid_mask

index = np.argsort(camera.images.integrated_minus_dark)  # sort by charge [min,...,max]
index = index[mask[index]]  # remove invalid items  & cam_module.invalid_mask
index = index[::-1]  # revers the order
print(index)

In [ ]:
camera.images.image2png(
    index=index, 
    directory="/data/straw_b/figures/", # add your preferred dir here
    f_name_formatter='bright_test_pic_{i}_{datetime}.png')  # save the images

In [ ]:
# The raw pixel values are NOT loaded by default to the module to save RAM.
# They can be accessed directly from the file with the index, index = None (default) to loads all images
a = camera.file_handler.raw.getunsorted([1,3])
print(a.shape)

In [ ]:
# The raw pixel values are NOT loaded by default to the module to save RAM.
# They can be accessed directly from the file with the index, index = None (default) to loads all images
a = camera.file_handler.raw.getunsorted(index=np.arange(len(cam_run.raw)))
print(a[0]) # returns array on default, even if only one element is accessed
print(a[0].shape) # 2D shape of picture
print(a.shape) # n_pic x 2D shape of picture

### Store the images with Lucifer enabled
You'd need to download a lot of data to do this, so be careful.

In [ ]:
import glob
from tqdm import tqdm

In [ ]:
search_str = f'{strawb.Config.raw_data_dir}/tumpmtspectrometer002/2021_06/TUMPMTSPECTROMETER002*20210601*-SDAQ-CAMERA.hdf5'
bar = tqdm(glob.glob(search_str), file=sys.stdout, unit='file')

In [ ]:
### loop over a couple of files
for filename in bar:
    bar.set_postfix({'file': filename})
    try:
        cam_run = camera.FileHandler(filename)
        picture_handler = camera.Images(cam_run)
    except Exception as err:
        print('error', filename, err)
        continue

    ### Store the 'brightest' images
    mode_list, mask_list = picture_handler.get_lucifer_mask()  # returns a mask for each lucifer setting
    mask_lucifer = np.any(mask_list, axis=0)  # compress the masks to one mask

    # mask over a threshold + mask invalid frames + mask lucifer (=LED) enabled
    mask = (picture_handler.integrated_minus_dark > 1e6) & picture_handler.invalid_mask & mask_lucifer
    if np.any(mask):
        print("found some!")
        index = np.argsort(picture_handler.integrated_minus_dark)  # sort by charge [min,...,max]
        index = index[mask[index]]  # remove invalid items  & cam_module.invalid_mask
        index = index[::-1]  # revers the order
        print(index)

        ## save just the first two pictures
        picture_handler.image2png(
            index=index[:2],
            directory="./figures/",
            f_name_formatter='lucifer_{i}_{datetime}.png'
        )  # save the images
        break

In [ ]:
### show one image here
plt.figure()
plt.imshow(picture_handler._rgb[0,:,:]/255)
plt.show()